In [1]:
import spectHR as cs
import pyhrv

In [2]:
DataSet = cs.SpectHRDataset("Example Data/SUB_002.xdf") 

In [3]:
DataSet = cs.borderData(DataSet)
#bDataSet.ecg = bDataSet.ecg.slicetime(500,2100)

In [4]:
DataSet = cs.filterECGData(DataSet, 
                          {'filterType': 'highpass', 
                           'cutoff': 1})

In [5]:
DataSet = cs.calcPeaks(DataSet)

In [6]:
%matplotlib widget
GUI = cs.spectplot(DataSet)

In [7]:
GUI

AppLayout(children=(HBox(children=(Select(class_='ma-2', color='primary', items=['Drag', 'Add', 'Find', 'Remov…